<a href="https://colab.research.google.com/github/Bayzid03/MNIST-Digit-Generator-GAN/blob/main/Building_Genrative_AI_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing Necessary Libraries and Dataset

In [2]:
!pip install tensorflow

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten
from tensorflow.keras.layers import BatchNormalization, LeakyReLU
from tensroflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam

import ssl
ssl._create_default_https_context = ssl._create_unverified_context
(X_train, _), (_, _) = mnist.load_data() # Keeping only the Train Images

Building a generator network which will transform a random noise vector into a data sample

In [ ]:
def build_generator():
    model = Sequential()
    model.add(Dense(256, input_dim=100))
    model.add(LeakyReLU(alpha=0.2)) # 'LeakyRelu' activation function used to introduce Non-Linearity into a NN model.
    model.add(BatchNormalization(momentum=0.8)) # 'BatchNormalization' layers help stablize training, improve convergence, and allow for higher learning rates by reducing internal covariate shift.
    model.add(Dense(512)) # Number of Neurons = 512
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8)) # 'momentum=0.8' controls how much past statistics (mean & variance) are remembered during training. A higher value (closer to 1) means updates are slower, ensuring more stable normalization.
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(784, activation='tanh')) # 'tanh' activation function inctroduce Non-Linearity and ensure outputs values in the range [-1, 1].
    model.add(Reshape((28, 28, 1)))
    return model

generator = build_generator()

Building a Discriminator which will classify input image as real or fake

In [ ]:
def build_discriminator():
    model = Sequential()
    model.add(Flatten(input_shape=(28, 28, 1))) # The Flatten layer converts Multidimentional input into 1D Vector so it can be fed into FC layer.
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid')) # 'sigmoid' function gives values between 0 and 1, representing a probability—often used to classify images as real or fake.
    return model

discriminator = build_discriminator()
discriminator.compile(optimizer=Adam(0.0002, 0.5), loss='binary_crossentropy', metrics=['accuracy'])

Combining Generator and Discriminator to train GenAI model to Generate Images

In [ ]:
discriminator.trainable = False # Freezing Discriminator bcz don't want to update the DCR weights while training the Generator.

# Chaining Generator + Discriminator into one combined GAN model.
gan_input = Input(shape=(100,))
generated_image = generator(gan_input)
gan_output = discriminator(generated_image)

gan = Model(gan_input, gan_output)
gan.compile(optimizer=Adam(0.0002, 0.5), loss='binary_crossentropy')

# Training Function
def train_gan(epochs, batch_size=128):
    X_train, _ = mnist.load_data()
    X_train = (X_train[0].astype(np.float32) - 127.5) / 127.5
    X_train = np.expand_dims(X_train, axis=3)

    real = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):
        idx = np.random.randint(0, X_train.shape[0], batch_size) # Sampling real MNIST images
        real_images = X_train[idx]

        # Generating fake images from random noise
        noise = np.random.normal(0, 1, (batch_size, 100))
        generated_images = generator.predict(noise)

        # Training Discriminator to distinguish real vs. fake
        d_loss_real = discriminator.train_on_batch(real_images, real)
        d_loss_fake = discriminator.train_on_batch(generated_images, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Training Generator to fool the Discriminator
        noise = np.random.normal(0, 1, (batch_size, 100))
        g_loss = gan.train_on_batch(noise, real)

        if epoch % 100 == 0:
            print(f"{epoch} [D loss: {d_loss[0]}, acc.: {100*d_loss[1]}] [G loss: {g_loss}]")
            save_images(epoch)

# This function is responsible for showing and saving the '5×5 image grid', and it's called every 100 epochs inside the training loop.
def save_images(epoch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, 100))
    generated_images = generator.predict(noise)

    generated_images = 0.5 * generated_images + 0.5

    fig, axs = plt.subplots(r, c)
    count = 0
    for i in range(r):
        for j in range(c):
            axs[i, j].imshow(generated_images[count, :, :, 0], cmap='gray')
            axs[i, j].axis('off')
            count += 1
    fig.savefig(f"gan_images_{epoch}.png")
    plt.close()

train_gan(epochs=10000, batch_size=64)